# Data loading, storage, and file formats

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [2]:
%pwd

u'd:\\inbox\\resources\\pydata-book-master'

## Reading and Writing Data in Text Format

In [3]:
list(open('ch06/ex1.csv'))

['a,b,c,d,message\n', '1,2,3,4,hello\n', '5,6,7,8,world\n', '9,10,11,12,foo']

In [4]:
df = pd.read_csv('ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_table('ch06/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
list(open('ch06/ex2.csv'))

['1,2,3,4,hello\n', '5,6,7,8,world\n', '9,10,11,12,foo']

In [7]:
pd.read_csv('ch06/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
#adds the names
pd.read_csv('ch06/ex2.csv', names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
names=['a','b','c','d','message']
pd.read_csv('ch06/ex2.csv',names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
list(open('ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [12]:
result = pd.read_table('ch06/ex3.txt', sep = '\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [13]:
list(open('ch06/ex4.csv'))

['# hey!\n',
 'a,b,c,d,message\n',
 '# just wanted to make things more difficult for you\n',
 '# who reads CSV files with computers, anyway?\n',
 '1,2,3,4,hello\n',
 '5,6,7,8,world\n',
 '9,10,11,12,foo']

In [14]:
pd.read_csv('ch06/ex4.csv',skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
list(open('ch06/ex5.csv'))

['something,a,b,c,d,message\n',
 'one,1,2,3,4,NA\n',
 'two,5,6,,8,world\n',
 'three,9,10,11,12,foo']

In [16]:
result = pd.read_csv('ch06/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [18]:
result = pd.read_csv('ch06/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
#assign null
sentinels = {'message':['foo','NA'],'something':['two']}
pd.read_csv('ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

In [20]:
result = pd.read_csv('ch06/ex6.csv')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
one      10000 non-null float64
two      10000 non-null float64
three    10000 non-null float64
four     10000 non-null float64
key      10000 non-null object
dtypes: float64(4), object(1)
memory usage: 390.7+ KB


In [21]:
#read specifically caused more efficient memory
pd.read_csv('ch06/ex6.csv',nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [22]:
chunker = pd.read_csv('ch06/ex6.csv',chunksize = 1000)
chunker

In [23]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [24]:
### Writing data out to text format
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [25]:
data.to_csv('ch06/out.csv')
list(open('ch06/out.csv'))

[',something,a,b,c,d,message\n',
 '0,one,1,2,3.0,4,\n',
 '1,two,5,6,,8,world\n',
 '2,three,9,10,11.0,12,foo\n']

In [26]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [27]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [28]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [29]:
data.to_csv(sys.stdout, index=False, cols=['a','b', 'c'])

something,a,b,c,d,message
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [30]:
dates = pd.date_range('1/1/2000',periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('ch06/tseries.csv')

In [31]:
list(open('ch06/tseries.csv'))

['2000-01-01,0\n',
 '2000-01-02,1\n',
 '2000-01-03,2\n',
 '2000-01-04,3\n',
 '2000-01-05,4\n',
 '2000-01-06,5\n',
 '2000-01-07,6\n']

In [32]:
Series.from_csv('ch06/tseries.csv',parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

In [33]:
### Manually Working with Delimited Formats
list(open('ch06/ex7.csv'))

['"a","b","c"\n', '"1","2","3"\n', '"1","2","3","4"\n']

In [35]:
#import csv
#f = open('ch06/ext7.csv')
#reader = csv.reader(f)
#
#for line in reader:
#    print line
#

In [36]:
#import csv
#%pwd
#f = open('d:/inbox/resources/pydata-book-master/ch06/ext7.csv')

#reader = csv.reader(f)
#for line in reader:
#    print line
#lines = list(csv.reader(open('ch06/ex7.csv')))
#header, values = lines[0], lines[1:]
#data_dict = {h: v for h, v in zip(header, zip(*values))}
#data_dict

### JSON Data

In [37]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [38]:
import json

In [39]:
result = json.loads(obj)
result

{u'name': u'Wes',
 u'pet': None,
 u'places_lived': [u'United States', u'Spain', u'Germany'],
 u'siblings': [{u'age': 25, u'name': u'Scott', u'pet': u'Zuko'},
  {u'age': 33, u'name': u'Katie', u'pet': u'Cisco'}]}

In [40]:
asjson = json.dumps(result)

In [41]:
siblings = DataFrame(result['siblings'], columns=['name','age'])
siblings

,name,age
0,Scott,25
1,Katie,33


### XML and HTML web Scrapping

In [42]:
# XML and HTML: web scraping
# Using the example from http://docs.python-guide.org/en/latest/scenarios/scrape/
# as the link from the book has changed and no longer works
from lxml import html
import requests

In [43]:
page = requests.get('http://econpy.pythonanywhere.com/ex/001.html')
tree = html.fromstring(page.content)
# We know there are buyers and prices from inspecting the html element
# This will create a list of buyers:
buyers = tree.xpath('//div[@title="buyer-name"]/text()')
# This will create a list of prices
prices = tree.xpath('//span[@class="item-price"]/text()')

In [44]:
print 'Buyers: ', buyers


Buyers:  ['Carson Busses', 'Earl E. Byrd', 'Patty Cakes', 'Derri Anne Connecticut', 'Moe Dess', 'Leda Doggslife', 'Dan Druff', 'Al Fresco', 'Ido Hoe', 'Howie Kisses', 'Len Lease', 'Phil Meup', 'Ira Pent', 'Ben D. Rules', 'Ave Sectomy', 'Gary Shattire', 'Bobbi Soks', 'Sheila Takya', 'Rose Tattoo', 'Moe Tell']


In [45]:
print 'Prices: ', prices


Prices:  ['$29.95', '$8.37', '$15.26', '$19.25', '$19.25', '$13.99', '$31.57', '$8.49', '$14.47', '$15.86', '$11.11', '$15.98', '$16.27', '$7.50', '$50.85', '$14.26', '$5.68', '$15.00', '$114.07', '$10.09']


### Binary Data Formats

In [46]:
frame = pd.read_csv('ch06/ex1.csv')
frame
frame.to_pickle('ch06/frame_pickle')

In [47]:
pd.read_pickle('ch06/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### reading xls

In [48]:
xls_file = pd.ExcelFile('ch06/data.xlsx')
table = xls_file.parse('Sheet1')
table

,variable,count
0,cats,34
1,dogs,26
2,mice,7
3,fish,14


## interacting with HTML and Web APIS

In [49]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
resp

<Response [200]>

In [50]:
data = json.loads(resp.text)
data[:5]

[{u'color': u'e10c02',
  u'name': u'Bug',
  u'url': u'https://api.github.com/repos/pydata/pandas/labels/Bug'},
 {u'color': u'4E9A06',
  u'name': u'Enhancement',
  u'url': u'https://api.github.com/repos/pydata/pandas/labels/Enhancement'},
 {u'color': u'FCE94F',
  u'name': u'Refactor',
  u'url': u'https://api.github.com/repos/pydata/pandas/labels/Refactor'},
 {u'color': u'75507B',
  u'name': u'Build',
  u'url': u'https://api.github.com/repos/pydata/pandas/labels/Build'},
 {u'color': u'3465A4',
  u'name': u'Docs',
  u'url': u'https://api.github.com/repos/pydata/pandas/labels/Docs'}]

In [51]:
issue_labels = DataFrame(data)
issue_labels

,color,name,url
0,e10c02,Bug,https://api.github.com/repos/pydata/pandas/lab...
1,4E9A06,Enhancement,https://api.github.com/repos/pydata/pandas/lab...
2,FCE94F,Refactor,https://api.github.com/repos/pydata/pandas/lab...
3,75507B,Build,https://api.github.com/repos/pydata/pandas/lab...
4,3465A4,Docs,https://api.github.com/repos/pydata/pandas/lab...
5,729FCF,Groupby,https://api.github.com/repos/pydata/pandas/lab...
6,06909A,Data IO,https://api.github.com/repos/pydata/pandas/lab...
7,8AE234,Visualization,https://api.github.com/repos/pydata/pandas/lab...
8,0b02e1,Indexing,https://api.github.com/repos/pydata/pandas/lab...
9,d7e102,Missing-data,https://api.github.com/repos/pydata/pandas/lab...


In [52]:
type(issue_labels)


pandas.core.frame.DataFrame

In [53]:
issue_labels['name']


0                   Bug
1           Enhancement
2              Refactor
3                 Build
4                  Docs
5               Groupby
6               Data IO
7         Visualization
8              Indexing
9          Missing-data
10          Performance
11            Reshaping
12    Difficulty Novice
13               Dtypes
14       Note To Selves
15           API Design
16      Error Reporting
17              Numeric
18               IO CSV
19               IO SQL
20                   CI
21            Internals
22            Timedelta
23            Frequency
24              Windows
25                Algos
26            Timezones
27               Period
28           MultiIndex
29          Categorical
Name: name, dtype: object

## interacting with databases

In [54]:

# Interact with databases
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [55]:
data = [('Atlanta', 'Georgia', 1.25, 6),
       ('Tallahassee', 'Florida', 2.6, 1),
       ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [56]:
con.executemany(stmt, data)
con.commit()

In [57]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[(u'Atlanta', u'Georgia', 1.25, 6),
 (u'Tallahassee', u'Florida', 2.6, 1),
 (u'Sacramento', u'California', 1.7, 5)]

In [58]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [59]:
DataFrame(rows, columns=zip(*cursor.description)[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,1
2,Sacramento,California,1.70,5


In [60]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,1
2,Sacramento,California,1.70,5


### storing and loading data in mongodb

In [61]:
#noyet